In [ ]:
# %pip install -qU pypdf langchain_community
# %pip install langchain_chroma langchain_openai langchain_text_splitters
# %pip install openai chromadb langchain pypdf
# %pip install langchainhub

### Uploading PDF

In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
import os

file_path = "../data/export.pdf"
loader = PyPDFLoader(file_path)

pages = loader.load()

print(len(pages))

12


### Splitting content

In [9]:
# Split Text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=20)
page_splits = text_splitter.split_documents(pages)
# TODO: Summarize each of these splits
# Then store them

In [ ]:
page_splits

In [10]:
len(page_splits)

31

### Pinecone Integration

In [ ]:
# %pip install pinecone_notebooks pinecone
# %pip install --upgrade --quiet  \
#     langchain-pinecone \
#     langchain-openai \
#     langchain \
#     langchain-community \
#     pinecone-notebooks

In [ ]:
# from pinecone_notebooks.colab import Authenticate

# Authenticate()
# # This is just to add API key. Actually we can skip this step if we have API key.

### Exporting Environments

In [11]:
from dotenv import load_dotenv
load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["PINECONE_API_KEY"] = pinecone_api_key

In [13]:
# Creating index
import time
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone()

# TODO: Get this index name from DB
index_name = "exporting-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

### Embed the splitted pages

In [14]:
from langchain_pinecone import PineconeVectorStore
embedding_model = OpenAIEmbeddings()
vectorstore = PineconeVectorStore.from_documents(page_splits, embedding_model, index_name=index_name)

In [16]:
query = "What is budgeting?"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

Expert  business advice, news, and trends, delivered weekly
By signing up you agree to the CO— Privacy Policy. You can opt out anytime.Developing a tech stack that supports customers and payments across
the world.
Adjusting to cultural di erences and language barriers.
Securing  nancing for exporting worldwide.
Estimating transportation and shipping costs.
Navigating problems with global freight networks.
Adhering to local and international laws.
Complying with trade agreements and regulations.
Finding foreign partners to address local commerce issues.


### Using the existing vectorstore

In [22]:
index_name = "lesson-1-index"  # Replace with your actual index name
vectorstore2 = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding_model)


In [23]:
query = "What is budgeting?"
docs = vectorstore2.similarity_search(query)
print(docs[0].page_content)

May 12, 2023, 08:15am EDT
GETTY
In the current economic climate, small-business owners face the
daunting task of navigating financial challenges while keeping their
ventures afloat. Trimming budgets has become a necessity for
many, as companies seek innovative ways to cut costs without
compromising their operations. While traditional cost-cutting


### Turn to Retriever

In [24]:
retriever = vectorstore.as_retriever(search_type="mmr")
retriever2 = vectorstore2.as_retriever(search_type="mmr")

In [ ]:
# found_docs = vectorstore.max_marginal_relevance_search(query, k=2, fetch_k=10)
# for i, doc in enumerate(found_docs):
#     print(f"{i + 1}.", doc.page_content, "\n")

In [ ]:
# matched_docs = retriever.invoke(query)
# for i, d in enumerate(matched_docs):
#     print(f"\n## Document {i}\n")
#     print(d.page_content)

### Creating RAG (not necessary for agent)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
# maximal marginal relevance
retriever = vectorstore.as_retriever(search_type="mmr")
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
topic_name = "Saving topics for small business owners"
results = rag_chain.invoke({"input": f"Create 10 topics for {topic_name}"})

print(results['answer'])

1. Implementing Energy-Efficient Practices
2. Utilizing Shared Office Spaces
3. Reducing Overhead Expenses
4. Renegotiating Contracts
5. Outsourcing Non-Core Activities
6. Leveraging Technology for Cost Savings
7. Adopting Remote Work Policies
8. Bulk Purchasing and Supplier Negotiations
9. Implementing Waste Reduction Strategies
10. Exploring Alternative Financing Options


In [ ]:
results = rag_chain.invoke({"input": f"""Now teach me the topics 1. Automating Manual Processes
2. Negotiating with Suppliers
3. Outsourcing Non-Core Tasks
4. Utilizing Available Space Efficiently
5. Refining and Documenting Processes
6. Leveraging No-Code Tools
7. Reducing Overhead Expenses
8. Renegotiating Contracts
9. Evaluating and Minimizing Waste
10. Engaging Team in Cost-Saving Solutions"""})

print(results['answer'])

Sure, here are brief explanations for each topic:

1. **Automating Manual Processes**: This involves using technology to perform repetitive tasks that were previously done manually. Tools like Zapier can automate data entry between systems, freeing up time for employees to focus on more strategic activities.

2. **Negotiating with Suppliers**: This strategy includes researching and comparing prices from different suppliers and then negotiating with your current suppliers to match or beat those prices. This can lead to better pricing and terms, ultimately reducing costs.

3. **Outsourcing Non-Core Tasks**: Outsourcing activities such as accounting, IT, or HR to specialized service providers can be more cost-effective than hiring full-time staff. This allows businesses to access expert resources without the associated employment costs.

4. **Utilizing Available Space Efficiently**: Making the most of your physical space can help reduce rent and utility costs. This might involve reorganiz

### Creating RAG as a tool

In [48]:
from langchain.tools.retriever import create_retriever_tool
tool1 = create_retriever_tool(
    retriever,  # The RAG chain instance you are using
    "BudgetingContentRAG",  # Renamed to better reflect the focus on learning materials
    "Search for information about any learning content. For any questions about learning content, you must use this tool!",
)

In [49]:
from langchain.tools.retriever import create_retriever_tool
tool2 = create_retriever_tool(
    retriever2,  # The RAG chain instance you are using
    "SavingRAG",  # Renamed to better reflect the focus on learning materials
    "Search for information about any learning content. For any questions about learning content, you must use this tool!",
)

### Create Agent

In [50]:
tools = [tool1, tool2]

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

### Custom Prompt

In [30]:
class AgentPrompts:
    agent_system_prompt = """
    You are an AI learning assistant helping a mentor coach their client on business-related matters.

    ```
    Client Personal Information:
    {client_personal_info}
    ```

    Client Business Information:
    {client_business_info}

    Topics to cover:
    {topics_to_cover}

    You are responsible for assessing the user's understanding of each topic. Use the RAG (Retrieval-Augmented Generation) tool, which has access to knowledge from an uploaded PDF file, to generate relevant content. The assistant should cover the topics sequentially, ensuring the user comprehends each topic before moving to the next one.

### Example Conversation Flow:

**Initial Conversation Starter:**
"Hello! Based on the client's business information and the topics we need to cover, I recommend starting with [specific topic]. How familiar are you with this topic?"

**Proceeding Based on Understanding:**

1. **Introduction to Topic:**
   - Provide a brief overview of the topic using content generated by the RAG tool.
   - Example: "Let's begin with an overview of [specific topic]. Here's a brief introduction: [content from RAG]."

2. **Assess Understanding:**
   - Ask questions to gauge the user's understanding.
   - Example: "Do you have any questions about this overview? Can you explain how this might apply to your business?"

3. **Deepen Understanding:**
   - Provide more detailed information or examples if the user needs more clarity.
   - Example: "Here are more details about [specific aspect]. [content from RAG]. How do you think this could impact your current business strategy?"

4. **Confirm Understanding:**
   - Ensure the user understands before moving on.
   - Example: "Great! Do you feel comfortable with this topic, or is there anything else you'd like to explore further?"

5. **Transition to Next Topic:**
   - Once understanding is confirmed, move to the next topic.
   - Example: "Since you're comfortable with [current topic], let's move on to [next topic]. Here's an overview: [content from RAG]."

This approach ensures a structured learning experience, allowing the user to build knowledge step-by-step while leveraging the content generated from the RAG tool.
"""



topics_generator_prompt = agent_system_prompt = """
You are an AI learning assistant helping a mentor coach their client on business-related matters.

```
Client Personal Information:
{client_personal_info}
```

Client Business Information:
{client_business_info}

Generate 10 topics for a small business owner.
The module name is: {module_name}
"""


### Topics Agent

In [51]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
custom_prompt_topic = ChatPromptTemplate.from_messages(
    [
        ("system", topics_generator_prompt),
        # MessagesPlaceholder("chat_history", optional=False),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)
custom_prompt_topic = custom_prompt_topic.partial(module_name="Saving Tips for Small Business Owners")

In [52]:
custom_prompt_topic

ChatPromptTemplate(input_variables=['agent_scratchpad', 'client_business_info', 'client_personal_info', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'module_name': 'Saving Tips for Small Business Owners'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['client_business_info', 'client_personal_info', 'module_name'], template='\nYou are an AI learning assistant helping a mentor coach their client on business-related matters.\n\n```\nClient Personal Information:\n{client_personal_info}\n```\n\nClient Business Information:\n{client_business_info}\n\nGenerate 10 topics for a small business owner.\nThe module name is: {module_name}\n')), HumanMessagePromptTempl

In [53]:
from langchain.agents import AgentExecutor, create_openai_tools_agent, load_tools
all_tools = tools
# Create an agent with all tools
topic_agent = create_openai_tools_agent(llm, all_tools, custom_prompt_topic)
# Create an agent executor by passing in the agent and tools
topic_agent_executor = AgentExecutor(agent = topic_agent, tools = all_tools, verbose = True, handle_parsing_errors=True)

In [55]:
user_input = "Generate 10 topic title"
response = topic_agent_executor.invoke({"input": user_input, 'client_personal_info':client_personal_info, 'client_business_info':client_business_info})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `SavingRAG` with `{'query': 'Cost-cutting strategies for small businesses'}`


1/22/24, 4:05 PM 17 Unconventional Money-Saving Tips For Small-Business Owners
https://www.forbes.com/sites/forbesbusinesscouncil/2023/05/12/17-unconventional-money-saving-tips-for-small-business-owners/?sh=79bdd1b9296f 4/7do it in-house, and it frees up the business to focus on its core
competencies. - Jeremiah Runyon , Superior Logistics Ohio
6. Implement Energy-Efﬁcient Practices
And Shared Ofﬁce Spaces
Small-business owners can consider implementing energy-efficient
practices to save on utilities, such as switching to LED lighting,
programmable thermostats and upgrading to energy-efficient

Chris Clear , Clear Storage Group, LLC
17. Focus On Reﬁnement And Solving
Problems
Two tactics that have never failed me are documenting existing
processes to see if process refinement can reduce costs and asking
my team how they would solve the problem. These two act

### Learning Agent

In [57]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
custom_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", AgentPrompts.agent_system_prompt),
        MessagesPlaceholder("chat_history", optional=False),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)
topics = response['output']
# topics = "[Open Topics]"

custom_prompt = custom_prompt.partial(topics_to_cover=topics)

In [59]:
print(response['output'])

Here are 10 topic titles for small business owners related to saving tips:

1. Cost-cutting strategies for small businesses
2. Financial planning tips for small business owners
3. Effective budgeting techniques for small businesses
4. Maximizing profits through efficient operations
5. Negotiation skills for reducing business expenses
6. Investment strategies for small business growth
7. Tax-saving methods for small business owners
8. Cash flow management in small businesses
9. Reducing overhead costs in a small business
10. Smart purchasing practices for small business sustainability


In [45]:
custom_prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'client_business_info', 'client_personal_info', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'topics_to_cover': '[Open Topics]'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['client_business_info', 'client_personal_info', 'topics_to_cover'], template='\n    You are an AI learning as

### Adding Memory

In [67]:
from langchain.memory import ConversationBufferWindowMemory, ConversationSummaryBufferMemory
from langchain.agents import AgentExecutor, create_openai_tools_agent, load_tools

memory = ConversationBufferWindowMemory(memory_key = "chat_history", return_messages = True, input_key = "input", max_token_limit=2000)

all_tools = tools
# print(all_tools)
# print(prompt)
# Create an agent with all tools
agent2 = create_openai_tools_agent(llm, all_tools, custom_prompt)
# Create an agent executor by passing in the agent and tools
agent_executor2 = AgentExecutor(agent = agent2, tools = all_tools, verbose = False, memory = memory, handle_parsing_errors=True)


In [36]:
client_personal_info = """
Name: Jane Doe
Age: 34
Education: MBA in Marketing
Experience: 10 years in small business management
Email: jane.doe@example.com
Phone: +1-234-567-8901
Location: Springfield, IL
"""

client_business_info = """
Business Name: Doe's Delights
Industry: Food and Beverage
Business Type: Small bakery specializing in organic and gluten-free products
Year Established: 2015
Number of Employees: 15
Annual Revenue: $500,000
Location: 123 Main Street, Springfield, IL

Business Summary:
Doe's Delights is a local bakery that has carved a niche in the organic and gluten-free segment. Founded in 2015 by Jane Doe, the business has steadily grown, catering to health-conscious consumers in Springfield and surrounding areas. The bakery offers a variety of baked goods, including bread, cakes, cookies, and pastries, all made with high-quality, organic ingredients.

Goals:
- Increase annual revenue by 20% over the next year
- Expand the product line to include vegan options
- Open a second location in the neighboring town
- Enhance online presence and e-commerce capabilities

Difficulties:
- Managing supply chain disruptions for organic ingredients
- Competition from larger bakeries and supermarkets
- Limited marketing budget
- Challenges in scaling up production without compromising quality
"""

## Initiator

This will be triggered once the user clicks 'Start' button.

In [ ]:
user_input = ""
response = agent_executor2.invoke({"input": user_input, 'client_personal_info':client_personal_info, 'client_business_info':client_business_info})
print(response['output'])

### Evaluation

In [66]:
user_input = "I have no idea."
response = agent_executor2.invoke({"input": user_input, 'client_personal_info':client_personal_info, 'client_business_info':client_business_info})
print(response['output'])

Let's delve into the topic of financial planning tips for small business owners. Here are some insights to help you understand this topic better:

1. **In-House Operations:** Consider handling certain operations in-house to save costs and focus on core competencies. This approach can lead to cost savings and operational efficiency.

2. **Energy-Efficient Practices:** Implementing energy-efficient practices like using LED lighting, programmable thermostats, and energy-efficient appliances can help reduce utility costs and improve sustainability.

3. **Shared Office Spaces:** Small businesses can explore shared office spaces as a cost-saving measure, reducing overhead expenses and fostering collaboration with other businesses.

4. **Investing in Software Tools and Automation:** Leveraging technology through software tools and automation can optimize operations, streamline workflows, reduce manual labor, eliminate redundancy, and cut down on expenses. Cloud-based accounting systems and pr